## Library imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from chromatography import *
from torch import optim, tensor
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mayavi import mlab
%matplotlib qt

## Dataset Import

In [3]:
alist = pd.read_csv('../data/Peterpeptides.csv')

## Useful functions definition

In [4]:
def step_decay(lr, iteration, num_episodes, steps=10, decay_factor=0.8):
    if iteration % (num_episodes // steps) == 0:
        return lr * decay_factor
    
    return lr

In [5]:
def loss_field(exp, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], [0.25, 3000000])
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

In [26]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=None)
pol = Policy(2, sigma_max = 0.2)
losses, best_program, mus, sigmas, n_pars, samples, mu_grads, sigma_grads = reinforce(
        exp, 
        pol, 
        [.25, 33.25], 
        num_episodes=2000, 
        batch_size=10,
        optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.0),
        lr=.5, 
        print_every=100,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=0.85),
        baseline=0.7,
        max_norm = 0.5,
        beta = 0
    )

Loss: 0.905209742323548, epoch: 100/2000
Loss: 0.7557500458482352, epoch: 200/2000
Loss: 1.0058576518815296, epoch: 300/2000
Loss: 0.7685605647653752, epoch: 400/2000
Loss: 0.8251742397359426, epoch: 500/2000
Loss: 0.7552848930445529, epoch: 600/2000
Loss: 0.8041727475219596, epoch: 700/2000
Loss: 0.7593390704340096, epoch: 800/2000
Loss: 0.7265015499718175, epoch: 900/2000
Loss: 0.7108945651573106, epoch: 1000/2000
Loss: 0.8599198104557281, epoch: 1100/2000
Loss: 0.7897539286111059, epoch: 1200/2000
Loss: 0.7518446406521563, epoch: 1300/2000
Loss: 0.7399275555384679, epoch: 1400/2000
Loss: 0.748343714932335, epoch: 1500/2000
Loss: 0.7189511040552441, epoch: 1600/2000
Loss: 0.7220820294569251, epoch: 1700/2000
Loss: 0.7162621998852153, epoch: 1800/2000
Loss: 0.7275718218324219, epoch: 1900/2000
Loss: 0.7077699927017145, epoch: 2000/2000


In [21]:
exp.reset()
exp.run_all(pol.mu.detach().numpy(), [0.25, 3000000])
exp.print_analytes(rc=(7, 5))
exp.loss()

0.6765882207361548

In [8]:
plt.plot(mus[:, 0], label='Mu: phi1')
plt.plot(mus[:, 1], label='Mu: phi2')
plt.legend()
plt.show()

In [9]:
plt.plot(sigmas[:, 0], label='Sigma: phi1')
plt.plot(sigmas[:, 1], label='Sigma: phi2')
plt.legend()
plt.show()

In [10]:
plt.plot(losses)

In [11]:
exp.reset()
exp.run_all(best_program.numpy(), [.25, 33.25])
exp.print_analytes(rc=(7, 5), title="Best Result")
exp.loss()

0.5193803066738278

In [12]:
X, Y, Loss_field = loss_field(exp, N = 100)

In [27]:
fig, ax = plt.subplots()
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

line, = plt.plot(mus[:, 0], mus[:, 1], color='k')
scat = plt.scatter(samples[:, 0], samples[:, 1], c='r', s=1)

def update(num, mu, samples, line, scat):
    line.set_data(mu[num-2:num, 0], mu[num-2:num, 1])
    line.axes.axis([0, 1, 0, 1])
    scat.set_offsets(samples[(num-2)*10: num*10 - 10, :])
    return line,

ani = animation.FuncAnimation(fig, update, len(mus[:, 0]), fargs=[mus, samples, line, scat],
                              interval=10, blit=False)
plt.show()

In [23]:
import matplotlib.patches as patches

fig, ax = plt.subplots()
ax.set_ylim(0, 1)
ax.set_xlim(0, 1)
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

patch = patches.Arrow(mus[0, 0], mus[0, 1], mu_grads[0, 0], mu_grads[0, 1], width=0.02, color='k')
scat1 = plt.scatter(samples[:, 0], samples[:, 1], c='r', s=5)
scat2 = plt.scatter(mus[:, 0], mus[:, 1], c='b', s=20)
def init():
    ax.add_patch(patch)
    return patch,

def update(num, mu, samples, scat1, scat2, ax):
    ax.patches.pop(0)
    patch = patches.Arrow(mus[num-2, 0], mus[num-2, 1], mu_grads[num-2, 0], mu_grads[num-2, 1], width=0.01, color='k')
    ax.add_patch(patch)
    scat1.set_offsets(samples[(num-2)*10: num*10, :])
    scat2.set_offsets(mu[num-2: num, :])
    return scat1,

ani = animation.FuncAnimation(fig, update, len(mus[:, 0]), fargs=[mus, samples, scat1, scat2, ax],
                              interval=1000, blit=False, init_func=init)


In [ ]:
N = -1
mlab.surf(X.T[:N, :N], Y.T[:N, :N], Loss_field.T[:N, :N])

In [ ]:
n=3

plt.scatter(samples[n*10:10+10*n ,0], samples[n*10:10+10*n,1], s=1.,c='r')
plt.arrow(mus[n,0], mus[n,1], mu_grads[n,0], mu_grads[n,1], length_includes_head=True, head_width=0.04, head_length=0.2)
plt.scatter(mus[n:n+2, 0], mus[n:n+2, 1])
plt.xlim((0, 1))
plt.ylim((0, 1))

In [ ]:
phis = np.linspace(0, 1, 1000)
losses = []
for phi1 in phis:
    exp.reset()
    exp.run_all([0.2, phi1], [0.25, 3000000])
    losses.append(exp.loss())
plt.plot(losses[:])

In [ ]:
plt.contourf(X, Y, Loss_field, levels=100)

In [ ]:
exp_ = ExperimentAnalytes(k0 = alist_.k0.values, S = alist_.S.values, h=0.001, grad='iso', run_time=None)
pol_ = Policy(2, sigma_max = 0.2)
losses_, best_program_, mus_, sigmas_, n_pars_ = reinforce(
        exp_, 
        pol_, 
        [.25, 33.25], 
        num_episodes=3000, 
        batch_size=10,
        optim=lambda a, b: torch.optim.SGD(a, b),
        lr=.1, 
        print_every=300,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=1.2),
        baseline=0.3
    )

In [ ]:
exp_.reset()
exp_.run_all(pol_.mu.detach().numpy(), [0.25, 3000000])
exp_.print_analytes(rc=(7, 5))
exp_.loss()

In [ ]:
plt.plot(mus_[:, 0], label='Mu: phi1')
plt.plot(mus_[:, 1], label='Mu: phi2')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas_[:, 0], label='Sigma: phi1')
plt.plot(sigmas_[:, 1], label='Sigma: phi2')
plt.legend()
plt.show()

In [ ]:
plt.plot(losses_)

In [ ]:
exp_.reset()
exp_.run_all(best_program_.numpy(), [.25, 33.25])
exp_.print_analytes(rc=(7, 5), title="Best Result")
exp_.loss()

In [ ]:
X_, Y_, Loss_field_ = loss_field(exp_, N = 100)

In [ ]:
fig, ax = plt.subplots()
plt.contourf(X_, Y_, Loss_field_, levels=100)
plt.colorbar()
line, = plt.plot(mus_[:, 0], mus_[:, 1], color='k')

def update(num, x, y, line):
    line.set_data(x[num-25:num], y[num-25:num])
    line.axes.axis([0, 1, 0, 1])
    return line,

ani = animation.FuncAnimation(fig, update, len(mus_[:, 0]), fargs=[mus_[:, 0], mus_[:, 1], line],
                              interval=1, blit=False)
plt.show()

In [ ]:
N = -1
mlab.surf(X_.T[:N, :N], Y_.T[:N, :N], Loss_field_.T[:N, :N])

In [ ]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=None)
pol = PolicyTime(2)
losses, best_program, mus, sigmas, n_par = reinforce_delta_tau(
        exp, 
        pol,
        num_episodes=5000, 
        batch_size=10,
        optim=torch.optim.SGD,
        lr=.2, 
        print_every=200,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=5, decay_factor=1.05),
        baseline=0.15,
    )
exp.print_analytes(rc=(7, 5))

In [ ]:
exp.reset()
exp.run_all(best_program[0], best_program[1])
exp.print_analytes(rc=(7, 5))
exp.loss()

In [ ]:
plt.plot(losses)

In [ ]:
plt.plot(mus[:, 0], label='Mu: 0')
plt.plot(mus[:, 1], label='Mu: 1')
plt.plot(mus[:, 2], label='Mu: 2')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas[:, 0], label='Sigma: 0')
plt.plot(sigmas[:, 1], label='Sigma: 1')
plt.plot(sigmas[:, 2], label='Sigma: 2')
plt.legend()
plt.show()